# Automatic Evaluator

In [37]:
%%capture
#########################
# Requirements and Imports
#########################
!pip install ipywidgets
!pip install widgetsnbextension

from os import path
import ipywidgets as widgets
from pathlib import Path

import torch
import numpy as np
import random

In [38]:
reproducibility_field = widgets.Checkbox(
    value = True,
    description = "Reproducibility",
    disabled = False
)
display(reproducibility_field)

Checkbox(value=True, description='Reproducibility')

In [39]:
reproducibility = reproducibility_field.value
if reproducibility:
    seed_field = widgets.IntText(
        value=0,
        placeholder=0,
        description="Seed:",
        disabled=False
    )
    display(seed_field)

IntText(value=0, description='Seed:')

In [40]:
# Set seeds for PRGs

reproducibility = reproducibility_field.value
if reproducibility:
    seed = seed_field.value
    print(seed)
    
    random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    # Use deterministic Algorithms
    torch.use_deterministic_algorithms(True)
    torch.utils.deterministic.fill_uninitialized_memory = True

0


## Game Selection

In [41]:
games = [x.name.replace('.py', '') for x in Path('../hackatari/games').glob('*.py') if x.is_file() and x.name != '__init__.py']
games = [ g.title() for g in games ] # Capitalize first letter

game_selector = widgets.Select(
    options = games,
    value = "Pong",
    description = 'Game:',
    disabled = False,
)
game_selector.layout.width = '30%'
display(game_selector)

Select(description='Game:', index=21, layout=Layout(width='30%'), options=('Amidar', 'Assault', 'Asterix', 'At…

In [42]:
selected_game = game_selector.value
selected_game

'Pong'

# Model Selection

In [43]:
models = [m for m in Path('../models').glob('**/*.gz') if m.is_file() and selected_game.lower() in str(m).lower()]

model_selector = widgets.Select(
    options=models,
    description="Select Model:",
    disabled=False
)
model_selector.layout.width = '30%'
display(model_selector)


Select(description='Select Model:', layout=Layout(width='30%'), options=(WindowsPath('../models/Pong/0/c51_cla…

In [44]:
model_path = model_selector.value
model_path

WindowsPath('../models/Pong/0/c51_classic_50M.gz')

# Add Run Arguments

In [45]:
human_player = widgets.HBox([
    widgets.Text(value='--human', description='Let user play', disabled=True),
    widgets.Dropdown(options=[True, False], value=False, description='Type:')
])

display(human_player)

In [46]:
human_opt = '-hu' if human_player.children[1].value else ''

In [47]:
mod_commands = ['lazy_enemy', 'up_drift']
modifies = widgets.VBox([
        widgets.HBox([
            widgets.Label(value=arg),
            widgets.Dropdown(options=[True, False], value=True, description='Value:')
        ], justify_content = 'space-between') for arg in mod_commands
    ])

display(modifies)

In [48]:
mod_args = ' '.join([f'{mod_commands[i]}' if modifies.children[i].children[1].value else '' for i in range(len(mod_commands))])
mod_args

'lazy_enemy up_drift'

# Run Game

In [49]:
eval_script = Path(r"scripts/eval.py")
project_root = path.dirname(path.abspath(""))
print(f"Project Root: {project_root}")

# use posix_paths compatibile with the OS
eval_path = (Path(project_root) / eval_script).as_posix() 
model_path = Path(model_path).as_posix()

print(f"Running Game: {selected_game} with model located at: {model_path}")


Project Root: f:\Studium\TUD_WS2024\LabKI\workspace\forked\HackAtari
Running Game: Pong with model located at: ../models/Pong/0/c51_classic_50M.gz


In [50]:

command = f"python {eval_path} -g {selected_game} -a {model_path} {human_opt} {mod_args}"
print("command: " + command)

!{command}

command: python f:/Studium/TUD_WS2024/LabKI/workspace/forked/HackAtari/scripts/eval.py -g Pong -a ../models/Pong/0/c51_classic_50M.gz  lazy_enemy up_drift
pygame 2.6.1 (SDL 2.28.4, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html
Loaded agent from ../models/Pong/0/c51_classic_50M.gz
Episode 1: Reward = -9.0
Episode 2: Reward = -5.0
Episode 3: Reward = -9.0
Episode 4: Reward = -8.0
Episode 5: Reward = -9.0
Episode 6: Reward = -2.0
Episode 7: Reward = 7.0
Episode 8: Reward = 1.0
Episode 9: Reward = -2.0
Episode 10: Reward = -1.0

Summary:
Agent: ../models/Pong/0/c51_classic_50M.gz
Total Episodes: 10
Average Reward: -3.70
Standard Deviation: 5.04
Min Reward: -9.0
Max Reward: 7.0
--------------------------------------


A.L.E: Arcade Learning Environment (version 0.10.2+c9d4b19)
[Powered by Stella]
Traceback (most recent call last):
  File "f:\Studium\TUD_WS2024\LabKI\workspace\forked\HackAtari\scripts\eval.py", line 143, in <module>
    main()
  File "f:\Studium\TUD_WS2024\LabKI\workspace\forked\HackAtari\scripts\eval.py", line 98, in main
    agent, policy = load_agent(agent_path, env, "cpu")
  File "f:\Studium\TUD_WS2024\LabKI\workspace\forked\HackAtari\venv\lib\site-packages\ocatari-2.1.0-py3.10.egg\ocatari\utils.py", line 257, in load_agent
    return agent, policy
UnboundLocalError: local variable 'agent' referenced before assignment


# Evaluation